# Trendvorhersage Metadaten

Die Vorhersage von Trends ist aus unterschiedlichen Gründen sehr interessant und wird schon seit sehr, sehr langer Zeit analysiert. 

Dafür kommen sowohl statistische Methoden als auch (modernere) Methoden des Maschinellen Lernens in Frage. Du wirst dir in diesem Teil beides anschauen und sowohl Trends in den Metadaten als auch in den Textbeiträgen selbst analysieren.

Zuerst betrachtest du Trends in Metadaten.

## Nutzung für die Reddit-Daten

Im Technology-Subreddit bzw. in dem daraus abgeleiteten Transport-Bereich gibt es verschiedene Metriken, die du betrachten kannst. Lade zuerst wie gewohnt die Daten in einen `DataFrame`.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f transport-all-comments.csv.xz || wget  https://github.com/heiseacademy/ml-python/raw/main/06-ml-projekt/transport-all-comments.csv.xz")

In [ ]:
import pandas as pd

posts = pd.read_csv("transport-all-comments.csv.xz", parse_dates=["created_utc"])

# damit du leichter addieren kannst
posts["count"] = 1

Um die Posts pro Tag auszurechnen, summierst du das einfach auf:

In [ ]:
posts_day = posts.set_index("created_utc")[["count"]].resample("D").sum()
posts_day

Schau dir die dazugehörige Grafik an:

In [ ]:
posts_day.plot.line()

Hier zeigt sich schon eine interessante Herausforderung: die Daten *fluktuieren*. Vermutlich ist der Grund eine Schwankung über die unterschiedlichen Wochentage. Das wirst du später noch genauer untersuchen, fürs Erste möchtest du die Anzahl der Posts auf Monatsbasis aggregrieren. Zum Glück geht das mit `pandas` ganz einfach:

In [ ]:
posts_month = posts.set_index("created_utc")[["count"]].resample("M").sum()
posts_month.plot()

Die Wahl der richtigen Datengranularität hat ganz erheblichen Einfluss auf die Trendvorhersage! Du hast die Daten nun stark geglättet und kannst langfristige Trends gut vorhersagen, die Unterschiede in den Wochentagen hast du damit aber *verwischt*. Sog. *Saisonalitäten* in den Jahreszeiten wären hingegen noch enthalten.

Versuche nun, hierfür die lineare Regression auszurechnen:

In [ ]:
from scipy.stats import linregress
lr = linregress(range(len(posts_month)), posts_month["count"].values)
lr

Zeichne die Daten ein, am besten geht das direkt als Vorhersage im `DataFrame`:

In [ ]:
import numpy as np
posts_month["count_trend"] = lr.intercept + lr.slope*np.arange(len(posts_month))
posts_month[["count", "count_trend"]].plot.line()

Das Ergebnis ist als Langzeittrend interpretierbar. Allerdings könnte der Anstieg auf auch die enorm große Postmenge Mitte 2022 zurückzuführen sein.

Für solch komplexe Szenarien gibt es noch bessere Verfahren. Damit könnte man z.B. auch Perioden etc. vorhersagen.

## Vorhersage mit Machine Learning

Du kannst auch kompliziertere Techniken des Maschinellen Lernens zur Trendvorhersage nutzen. Häufig wird dazu ein sog. [LSTM](https://de.wikipedia.org/wiki/Long_short-term_memory) verwendet, allerdings ist das ziemlich kompliziert und erfordert auch lange Rechenzeiten. Es muss auch einfacher gehen!

## Vorhersage mit `prophet`

[`prophet`](https://facebook.github.io/prophet/) ist ein Softwarepaket von Facebook, das auf die Trendvorhersage spezialisiert ist. Es ist sehr einfach bedienbar und nutzt fortgeschrittene Methoden zur Vorhersage.

`prophet` produziert zwar nicht immer die besten Resultate, aber man muss sich schon sehr anstrengen und ganz spezielle Software verwenden, um es im Einzelfall zu schlagen.

Deswegen betrachtest du nun `prophet` genauer und wendest es auf unterschiedliche Szenarien an.

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

`prophet` erfordert einen `DataFrame`, der aus den beiden Spalten `ds` und `y` besteht:

In [ ]:
df = pd.DataFrame({"ds": posts_month.index.values, 
                   "y": posts_month["count"].values})
df

Zuerst muss du ein `Prophet`-Objekt instanziieren, um anschließend die `fit()`-Methode aufrufen zu können.

In [ ]:
m = Prophet()
m.fit(df)

Anschließend kannst du einen neuen `DataFrame` erzeugen, der die Zeitpunkte für die Vorhersage enthält. Du möchtest die Vorhersage auf Monatsbasis durchführen:

In [ ]:
future = m.make_future_dataframe(periods=24, freq='M')
future

... und schließlich auch berechnen:

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Eine Stärke von `prophet` sind auch die integrierten Funktionen zur Visualisierung:

In [ ]:
fig1 = m.plot(forecast)

Das Ergebnis sieht schon sehr gut aus, viele Punkte befinden sich innerhalb des *Konfidenzkorridors*. Die Ausrutscher hat `prophet` als solche behandelt und ignoriert.

Spannend sind auch die Komponenten der Vorhersage, die nur den Trend und die Saisonalität darstellen:

In [ ]:
fig2 = m.plot_components(forecast)

## Unterschiedliche Wochentage 

Nun kannst du dich fragen, ob die Verteilung der Posts über die Wochentage auch unterschiedlich ist. Dazu musst du zurückgehen auf die noch nicht aggregierten Daten. Der Rest ist fast identisch zu dem oberen Verfahren:

In [ ]:
df = pd.DataFrame({"ds": posts_day.index, "y": posts_day["count"]})

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(df)

*fast*... Hier wird der Future-`DataFrame` auf Tagesbasis erzeugt für zwei Jahre (Schaltjahr!) in die Zukunft:

In [ ]:
future = m.make_future_dataframe(periods=731, freq='D')
future

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

Das *Geflimmere* wird schon ziemlich gut abgebildet.

Betrachte nun die Komponenten:

In [ ]:
fig2 = m.plot_components(forecast)

Offenbar wird der Transport-Flair des  Technology-Reddit im Juni mehr verwendet als im Dezember. Samstag und Sonntag sind Tage mit ziemlich geringer Nutzung.